In [1]:
import os
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from time import sleep

from datetime import datetime 

In [20]:
url_base = 'https://api.yelp.com/v3/'

url_review = 'businesses/{id}/reviews'
url_business = 'businesses/search'
highest_reviews = '?sort_by=rating_desc'
lowest_reviews = '?sort_by=rating_asc'

In [3]:
apikey = 'Bearer CrgilvjOJW-U-DnMiyFz_xypRqtVtfL_yekfgb4Kg5xNRWe1A83s2llrJ7RxnXYa0iqT8GwTCHJZKvYMHL7CmV1zJPe0D1_jWrH8W0bMxTtR6Ufksbg5REDVrSDbY3Yx'

**get business info**

In [5]:
url = os.path.join(url_base, url_business)
api_header = {'Authorization': apikey, 'accept': 'application/json'}

for offset in range(0, 50, 50):
    params = {
        'location': 'San Francisco',
        'term': 'Daeho',
        'limit': 50,
        'offset': offset
    }

    r = requests.get(url, headers=api_header, params=params)
    
    result = json.loads(r.text)
    busi_list = result['businesses']
    busi_df = pd.DataFrame(columns=['id', 'name', 'rating', 'url', 'review_cnt'])
    for i in range(len(busi_list)):
        busi_df.loc[i, 'id'] = busi_list[i]['id']
        busi_df.loc[i, 'name'] = busi_list[i]['name']
        busi_df.loc[i, 'rating'] = busi_list[i]['rating']
        busi_df.loc[i, 'url'] = busi_list[i]['url'].split('?')[0]
        busi_df.loc[i, 'review_cnt'] = busi_list[i]['review_count']
    
    result = pd.DataFrame(columns=['id', 'name', 'rating', 'review', 'total_review'])
    idx = 0
    error_count = 0 
    for res in range(len(busi_df)):
        new_page = "?start={}"
        base_url = busi_df.loc[res, 'url']
        cnt = busi_df.loc[res, 'review_cnt']

        # soup = BeautifulSoup(content, "html.parser")
        sleep(2)
        for i in range(0, min(cnt, 100), 10):
            new_page_url = base_url + new_page.format(i)
            try: 
                new_content = requests.get(new_page_url).content
            except Exception as e:
                with open('logs.txt','w') as f:
                    timestamp = str(datetime.now())
                    error_string = f'{timestamp} : Error while using request. Error output {e} \n'
                    f.write(error_string)
                    error_count += 1

            try:

                new_soup = BeautifulSoup(new_content, "html.parser")

                relevant= new_soup.find_all('p', class_='comment__09f24__gu0rG css-qgunke')
                for div in relevant:
                    for html_class in div.find_all('span',class_="raw__09f24__T4Ezm"):
                        text = html_class.find('span')
                        review = html_class.getText()
                        result.loc[idx, 'id'] = busi_df.loc[res, 'id']
                        result.loc[idx, 'name'] = busi_df.loc[res, 'name']
                        result.loc[idx, 'rating'] = busi_df.loc[res, 'rating']
                        result.loc[idx, 'total_review'] = busi_df.loc[res, 'review_cnt']
                        result.loc[idx, 'review'] = review

                        idx += 1
            except Exception as e:
                with open('logs.txt','w') as f:
                    timestamp = str(datetime.now())
                    error_string = f'{timestamp} : Error while parsing with BeautifulSoup. Error output {e} \n'
                    f.write(error_string)
                    error_count += 1

            if error_count > 10:
                break
            
    outfile = str(offset)+'_review_result.csv'
    result.to_csv(outfile)


KeyboardInterrupt: 

In [29]:
url = os.path.join(url_base, url_business)
api_header = {'Authorization': apikey, 'accept': 'application/json'}


offset = 1

params = {
    'location': 'San Francisco',
    'term': 'Daeho',
    'limit': 1,
    'offset': offset
}

r = requests.get(url, headers=api_header, params=params)
result = json.loads(r.text)
busi_list = result['businesses']
busi_df = pd.DataFrame(columns=['id', 'name', 'rating', 'url', 'review_cnt'])


In [50]:
i=0
busi_df.loc[i, 'id'] = busi_list[i]['id']
busi_df.loc[i, 'name'] = busi_list[i]['name']
busi_df.loc[i, 'rating'] = busi_list[i]['rating']
busi_df.loc[i, 'url'] = busi_list[i]['url'].split('?')[0]
busi_df.loc[i, 'review_cnt'] = busi_list[i]['review_count']


result = pd.DataFrame(columns=['id', 'name', 'rating', 'review','highest_review', 'lowest_review','total_review'])
idx = 0
error_count = 0 
res = 1

In [64]:
res= 0
new_page = "?start={}"
base_url = busi_df.loc[res, 'url']
highest_reviews_url = base_url + highest_reviews
lowest_reviews_url = base_url + lowest_reviews
cnt = busi_df.loc[res, 'review_cnt']

# soup = BeautifulSoup(content, "html.parser")
sleep(2)
for i in range(0, min(cnt, 100), 10):
    new_page_url = base_url + new_page.format(i)
    new_page_url_highest = highest_reviews_url + new_page.format(i)
    new_page_url_lowest = lowest_reviews_url + new_page.format(i)
    new_content = requests.get(new_page_url).content
    new_content_highest = requests.get(new_page_url_highest).content
    new_content_lowest = requests.get(new_page_url_lowest).content

    new_soup = BeautifulSoup(new_content, "html.parser")
    new_soup_highest = BeautifulSoup(new_content_highest, "html.parser")
    new_soup_lowest = BeautifulSoup(new_content_lowest, "html.parser")
    relevant= new_soup.find_all('p', class_='comment__09f24__gu0rG css-qgunke')
    relevant_highest = new_soup_highest.find_all('p', class_='comment__09f24__gu0rG css-qgunke')
    relevant_lowest = new_soup_lowest.find_all('p', class_='comment__09f24__gu0rG css-qgunke')


In [75]:
highest_reviews_url

'https://www.yelp.com/biz/daeho-kalbijjim-and-beef-soup-san-francisco-10?sort_by=rating_desc'

In [73]:
new_soup_lowest

<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/no-js/,"js");</script><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="en-US" http-equiv="Content-Language"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><link content="#FF1A1A" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" rel="mask-icon" sizes="any"/><link href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico" rel="shortcut icon"/><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>
            window.yelp = window.yelp || {};
      

In [71]:
new_soup_highest == new_soup_lowest

False

In [70]:
relevant_lowest == relevant_highest

True

In [52]:
for div in relevant:
    for html_class in div.find_all('span',class_="raw__09f24__T4Ezm"):
        text = html_class.find('span')
        review = html_class.getText()
        result.loc[idx, 'id'] = busi_df.loc[res, 'id']
        result.loc[idx, 'name'] = busi_df.loc[res, 'name']
        result.loc[idx, 'rating'] = busi_df.loc[res, 'rating']
        result.loc[idx, 'total_review'] = busi_df.loc[res, 'review_cnt']
        result.loc[idx, 'review'] = review

        idx += 1
idx=0
for div in relevant_highest:
    for html_class in div.find_all('span',class_="raw__09f24__T4Ezm"):
        text = html_class.find('span')
        review = html_class.getText()
        result.loc[idx, 'highest_review'] = review
        idx+=1
idx=0
for div in relevant_lowest:
    for html_class in div.find_all('span',class_="raw__09f24__T4Ezm"):
        text = html_class.find('span')
        review = html_class.getText()
        result.loc[idx, 'lowest_review'] = review
        idx+=1


#outfile = str(offset)+'_review_result_test.csv'
#result.to_csv(outfile)


In [53]:
result

,id,name,rating,review,highest_review,lowest_review,total_review
0,CBeM82Cl4Lxf4cmNOtGA-w,Daeho Kalbijjim & Beef Soup,4.5,Wow! We ordered the #2 Braised Beef Rib and Ox...,I was craving bibimbap the other day and was r...,I was craving bibimbap the other day and was r...,136
1,CBeM82Cl4Lxf4cmNOtGA-w,Daeho Kalbijjim & Beef Soup,4.5,Was doing some grocery shopping until I ran in...,I love that there's no long wait and you recei...,I love that there's no long wait and you recei...,136
2,CBeM82Cl4Lxf4cmNOtGA-w,Daeho Kalbijjim & Beef Soup,4.5,Ohh sooooo good. In the beginning was a bit co...,My bf and I were showing my friend from out of...,My bf and I were showing my friend from out of...,136
3,CBeM82Cl4Lxf4cmNOtGA-w,Daeho Kalbijjim & Beef Soup,4.5,DELIVERY order of Kalbijjim was AMAZING. Plent...,This location is just as good as the Japantown...,This location is just as good as the Japantown...,136
4,CBeM82Cl4Lxf4cmNOtGA-w,Daeho Kalbijjim & Beef Soup,4.5,"Love the Kalbi Jjim, a bit pricy but more than...","The main message I want to say is, if you've n...","The main message I want to say is, if you've n...",136
5,CBeM82Cl4Lxf4cmNOtGA-w,Daeho Kalbijjim & Beef Soup,4.5,So good! I've been a huge fan of Sun Non Dang ...,I enjoyed the beef rib soup here with the appe...,I enjoyed the beef rib soup here with the appe...,136
6,CBeM82Cl4Lxf4cmNOtGA-w,Daeho Kalbijjim & Beef Soup,4.5,"Bring your vaccine card, you have to present y...",I had the kalbitang (beef rib soup) and it hit...,I had the kalbitang (beef rib soup) and it hit...,136
7,CBeM82Cl4Lxf4cmNOtGA-w,Daeho Kalbijjim & Beef Soup,4.5,I gotta say the hype around this place is real...,I didn't realize this place was in a food cour...,I didn't realize this place was in a food cour...,136
8,CBeM82Cl4Lxf4cmNOtGA-w,Daeho Kalbijjim & Beef Soup,4.5,Food is great. The beef soup is a must try. Ve...,Love coming here for their braised beef & oxta...,Love coming here for their braised beef & oxta...,136
9,CBeM82Cl4Lxf4cmNOtGA-w,Daeho Kalbijjim & Beef Soup,4.5,Stopped in to order the infamous kalbijim. De...,"I love this secret daeho, compare to other dae...","I love this secret daeho, compare to other dae...",136


In [ ]:
result = json.loads(r.text)
len(result['businesses'])

In [ ]:
busi_list = result['businesses']
busi_list[0]

In [ ]:
busi_df = pd.DataFrame(columns=['id', 'name', 'rating', 'url', 'review_cnt'])
for i in range(len(busi_list)):
    busi_df.loc[i, 'id'] = busi_list[i]['id']
    busi_df.loc[i, 'name'] = busi_list[i]['name']
    busi_df.loc[i, 'rating'] = busi_list[i]['rating']
    busi_df.loc[i, 'url'] = busi_list[i]['url'].split('?')[0]
    busi_df.loc[i, 'review_cnt'] = busi_list[i]['review_count']

In [ ]:
busi_df

In [ ]:
result = pd.DataFrame(columns=['id', 'name', 'rating', 'review', 'total_review'])
idx = 0
error_count = 0 
for res in range(len(busi_df)):
    new_page = "?start={}"
    base_url = busi_df.loc[res, 'url']
    cnt = busi_df.loc[res, 'review_cnt']

    # soup = BeautifulSoup(content, "html.parser")
    sleep(2)
    for i in range(0, min(cnt, 100), 10):
        new_page_url = base_url + new_page.format(i)
        try: 
            new_content = requests.get(new_page_url).content
        except Exception as e:
            with open('logs.txt','w') as f:
                timestamp = str(datetime.now())
                error_string = f'{timestamp} : Error while using request. Error output {e} \n'
                f.write(error_string)
                error_count += 1
        
        try:
            
            new_soup = BeautifulSoup(new_content, "html.parser")

            relevant= new_soup.find_all('p', class_='comment__09f24__gu0rG css-qgunke')
            for div in relevant:
                for html_class in div.find_all('span',class_="raw__09f24__T4Ezm"):
                    text = html_class.find('span')
                    review = html_class.getText()
                    result.loc[idx, 'id'] = busi_df.loc[res, 'id']
                    result.loc[idx, 'name'] = busi_df.loc[res, 'name']
                    result.loc[idx, 'rating'] = busi_df.loc[res, 'rating']
                    result.loc[idx, 'total_review'] = busi_df.loc[res, 'review_cnt']
                    result.loc[idx, 'review'] = review

                    idx += 1
        except Exception as e:
            with open('logs.txt','w') as f:
                timestamp = str(datetime.now())
                error_string = f'{timestamp} : Error while parsing with BeautifulSoup. Error output {e} \n'
                f.write(error_string)
                error_count += 1
                
        if error_count > 10:
            break
            


In [ ]:
result.to_csv('50_review_result.csv')

In [ ]:
result

In [13]:
import pandas as pd
import os

In [14]:
filelists = os.listdir('data')
# result = pd.DataFrame()
# for f in filelists:
#     tmp = pd.read_csv('data/'+f)

In [17]:
result = pd.concat([pd.read_csv('data/'+f) for f in filelists ])

In [18]:
result.to_csv('data.csv')